In [1]:
import argparse
import os

import numpy as np
import tensorflow as tf

from dataset import KinQueryDataset, preprocess

In [2]:
def _batch_loader(iterable, n=1):
    """
    데이터를 배치 사이즈만큼 잘라서 보내주는 함수입니다. PyTorch의 DataLoader와 같은 역할을 합니다

    :param iterable: 데이터 list, 혹은 다른 포맷
    :param n: 배치 사이즈
    :return:
    """
    length = len(iterable)
    for n_idx in range(0, length, n):
        yield iterable[n_idx:min(n_idx + n, length)]


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [3]:
# User options
batch = 1
epochs = 1

embedding = 30
strmaxlen = 200
DATASET_PATH = '../sample_data/movie_review/'

# 모델의 specification
input_size = embedding*strmaxlen
output_size = 10
hidden_layer_size = 200
learning_rate = 0.001
character_size = 251

x = tf.placeholder(tf.int32, [None, strmaxlen])
y_ = tf.placeholder(tf.float32, [None, output_size])
# 임베딩
char_embedding = tf.get_variable('char_embedding', [character_size, embedding])
embedded = tf.nn.embedding_lookup(char_embedding, x)

# 첫 번째 레이어
cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer_size)
cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.9)
#tf.cast(cell, tf.int32)
#tf.cast(x, tf.int32)

output, state = tf.nn.dynamic_rnn(cell, embedded, dtype=tf.float32)
output = tf.transpose(output, [1, 0, 2])
output = output[-1]
print("output shaep",output.shape)

# 두 번째 (아웃풋) 레이어
second_layer_weight = weight_variable([hidden_layer_size, output_size])
second_layer_bias = bias_variable([output_size])
output = tf.matmul(output, second_layer_weight) + second_layer_bias
#output_sigmoid = tf.sigmoid(output)
print("output shaep",output.shape)
print("lables shaep",y_.shape)
# loss와 optimizer
#global_step = tf.Variable(0)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y_))
#ls = tf.reduce_mean(tf.losses.mean_squared_error(labels=y_, predictions=output))
ls = tf.reduce_mean(tf.square(output-y_))

#print(loss)

#learning_rate= tf.train.exponential_decay(learning_rate, global_step, 10000, 0.75)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(ls)
#train_step = tf.train.AdadeltaOptimizer(1.0, 0.95, 1e-6).minimize(cost)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

output shaep (?, 200)
output shaep (?, 10)
lables shaep (?, 10)


In [4]:
dataset = KinQueryDataset(DATASET_PATH, strmaxlen)
dataset_len = len(dataset)

In [5]:
print(dataset_len)

107


In [6]:
np.eye(2)[[0]]

array([[ 1.,  0.]])

In [7]:
def label_one_hot(labels):
    one_hot = (np.arange(output_size) == labels[:]).astype(np.int32)
    return one_hot

In [8]:
label_one_hot([1])

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [9]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()
#batch = 1
dataset = KinQueryDataset(DATASET_PATH, strmaxlen)
dataset_len = len(dataset)
one_batch_size = dataset_len//batch
if dataset_len % batch != 0:
    one_batch_size += 1
# epoch마다 학습을 수행합니다.
for epoch in range(epochs):
    avg_loss = 0.0
    avg_l=0.0
    for i, (data, labels) in enumerate(_batch_loader(dataset, batch)):
        #zero = np.zeros([batch,1])
        #zero[0] = 
        _, loss, l,o = sess.run([train_step, cost, ls,output],
                           feed_dict={x: data, y_: label_one_hot(labels)})
        if math.isnan(loss):
            print('Detected NaN')
            import pdb; pdb.set_trace()
        #print('Batch : ', i + 1, '/', one_batch_size,', BCE in this minibatch: ', (loss))
        avg_loss += float(loss)
        avg_l += float(l)
        print(o)
    print('epoch:', epoch, ' train_loss:', float(avg_loss/one_batch_size),float(avg_l/one_batch_size),timeSince(start))

[[ 0.09317937  0.06905934  0.14501873  0.06492354  0.08705307  0.09487598
   0.11142266  0.14549312  0.13151973  0.12215553]]
[[ 0.02473442  0.07149015  0.12236015  0.06597787  0.05370649  0.19037379
   0.10811619  0.09205828  0.0746674   0.11529785]]
[[ 0.0391154   0.03739097  0.0912198   0.06758422  0.04401446  0.19891334
   0.09024289  0.15181908  0.07992584  0.11402007]]
[[ 0.01690291  0.04172587  0.05041428  0.02973522  0.0179518   0.22341949
   0.06588715  0.18100372  0.04418751  0.08373076]]
[[ 0.00582891  0.02172139  0.03172526  0.05002946 -0.01384088  0.18829924
  -0.00545477  0.17201346  0.05207273  0.02130491]]
[[ 0.00393488 -0.02390866 -0.01511571 -0.00306205  0.01454236  0.16549426
  -0.04366522  0.1824078   0.01956982  0.00744063]]
[[-0.0348107  -0.04801531 -0.04563063 -0.01182494 -0.01058286  0.13394311
  -0.07029643  0.17144272  0.01433792 -0.02799732]]
[[-0.03786    -0.01621224 -0.01772789  0.01060659  0.01339936  0.10779733
  -0.06315109  0.14366224  0.05002035 -0.022

[[-0.00990265  0.0005295   0.01374606 -0.01547445  0.0175954  -0.0046103
  -0.02241769 -0.02076409  0.00302655  0.01530813]]
[[ 0.01053747  0.00047559 -0.00045329  0.01075107  0.02697458 -0.00793052
   0.00570326 -0.0051332  -0.00076134 -0.00760055]]
[[ 0.0308515   0.05110308 -0.00270513  0.02360534  0.01889315  0.01612415
  -0.0266325   0.01134675  0.03443271 -0.01453704]]
[[ 0.00148254  0.09545395  0.0092511   0.03173648  0.05575669  0.02547744
  -0.03007405  0.00522115  0.12610182 -0.02363237]]
[[-0.01311142  0.08010703  0.0126396   0.01716281  0.03775664  0.00462386
  -0.0072839   0.01623014  0.15433212 -0.02356572]]
[[-0.0214399   0.09894601  0.01214742  0.02342603  0.05990922  0.02758168
  -0.00587254  0.01315174  0.18704978 -0.0131316 ]]
[[-0.00339919  0.09692983  0.03580248  0.01267479  0.05593323  0.02733994
  -0.01522077  0.02134204  0.20509648 -0.00734536]]
[[-0.00699548  0.11072253  0.01846363  0.02339452  0.05103736  0.01435227
  -0.00290303  0.01603802  0.21825284 -0.0090

In [10]:
#ze = np.zeros([batch,2])
#ze[0,0] = 1
ze = np.zeros([2])
ze[0] = 1

print(ze)

[ 1.  0.]


In [11]:


for i, (data, labels) in enumerate(_batch_loader(dataset, batch)):
    #t = tf.nn.softmax_cross_entropy_with_logits(
    #            logits=ze, labels=one(labels))
    for i in range(batch):
        print(labels[i])
    #print(one(labels), ze, t.eval())

[ 5.]
[ 7.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 1.]
[ 6.]
[ 9.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 1.]
[ 8.]
[ 8.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 9.]
[ 10.]
[ 10.]
[ 0.]
[ 1.]
[ 1.]
[ 1.]
[ 1.]
[ 3.]
[ 5.]
[ 7.]
[ 8.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 1.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]
[ 10.]


In [12]:
o=np.array([[0]])
o[0,0]
idx = np.argmax(o,axis=0)
print(o[idx])

[[0]]


In [13]:
n = np.ndarray([,2])


SyntaxError: invalid syntax (<ipython-input-13-5a677b5d0595>, line 1)